In [1]:
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import random

from helpers import imshow

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [2]:
# Insert credentials and initialize Face-API
subscription_key = "" #Here you have to paste your primary key
assert subscription_key
face_api_url = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect'

In [3]:
# Parameters required for the request
headers = {
     'Content-Type': 'application/octet-stream',
     'Ocp-Apim-Subscription-Key': subscription_key,
}
params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion',
#     'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}

## Helper functions

In [4]:
def processRequest(bytes_string, face_api_url = face_api_url, headers = headers, params = params):
    response = requests.post(face_api_url, params=params, headers=headers, data = bytes_string)
    faces = response.json()
    return faces

In [5]:
def getEmotion(result):        
    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        emotions = currFace['faceAttributes']['emotion']
        currEmotion = max(emotions.items(), key=operator.itemgetter(1))[0]
    return currEmotion

# Demo Game: Mimic Emoji

### Method that imports a library of emoji

In [6]:
import glob
def importEmoji():
    emoji_dict = {}
    images = glob.glob('emoji_images/*.jpg')    
    for fname in images:
        name = fname.split("/")[-1].split(".")[0]
        emoji_img = cv2.imread(fname, -1)
        emoji_img = convert_to_bgra(emoji_img)
        emoji_dict[name] = emoji_img
    return emoji_dict

### Method that imports utility images

In [7]:
def importUtil():
    util_dict = {}
    images = glob.glob('util_img/*.jpg')    
    for fname in images:
        name = fname.split("/")[-1].split(".")[0]
        util_img = cv2.imread(fname)
        util_img = convert_to_bgra(util_img)
        util_dict[name] = util_img
    return util_dict

### Method that converts image from BGR to BGRA

In [8]:
# Method that converts image from BGR to BGRA
def convert_to_bgra(img):
    # If image is already BGRA, just return it
    if img.shape[2] == 4:
        return img
    
    # Subtract emoji contour
    mask = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(mask, 230, 255, cv2.THRESH_BINARY_INV)
    _, contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    areas = [cv2.contourArea(cont) for cont in contours] # Calculate areas
    cnt = contours[np.argmax(areas)] # Only keep the largest contour

    mask_cnt = np.zeros(img.shape[:2], np.uint8)
    cv2.drawContours(mask_cnt, [cnt], -1, 255, -1)

    # Convert to BGRA
    b_channel, g_channel, r_channel = cv2.split(img)
    alpha_channel = mask_cnt
    img_BGRA = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))
    return img_BGRA

### Method that augments image on the video frame

In [9]:
# Augment image to a frame
def augment_on_frame(frame, img, x_offset, y_offset, x_size, y_size):
    # set size
    img = cv2.resize(img, (x_size,y_size), interpolation = cv2.INTER_AREA)
  
    # set offset
    y1, y2 = y_offset, y_offset + img.shape[0]
    x1, x2 = x_offset, x_offset + img.shape[1]

    alpha_s = img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    try:
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_s * img[:, :, c] +
                                      alpha_l * frame[y1:y2, x1:x2, c])
    except:
        return None
    return frame

### Game - version1

In [164]:
camera = cv2.VideoCapture(0)
# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
camera.set(cv2.CAP_PROP_FPS, 24)

# Import emoji
emoji_dict = importEmoji()
current_emoji = random.choice(emoji_dict.keys())

# Import utility images
util_dict = importUtil()

# Initialize score and counter for displaying result information
i = 0
score = 0

while True:
    i -= 1
    if i <= 0:
        flag = False
    
    
    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    
    # Augment frame with emoji
    emoji_img = emoji_dict[current_emoji]
    augment_on_frame(frame, emoji_img, 10, 10, 150, 150)
    
    if cv2.waitKey(5) == 32:
        # If space bar is pressed, extract imotion
        bytes_string = cv2.imencode('.jpg', frame)[1].tostring()
        result = processRequest(bytes_string)
        
        
        if result is not None and len(result) > 0:    
            # Detect Emoji
            data8uint = np.fromstring( bytes_string, np.uint8 ) # Convert string to an unsigned int array
            img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
            detected_emotion = getEmotion( result )
                        
            # Check if emoji is matched
            if detected_emotion == current_emoji:
                guess = "Correct!"
                correctness_img = util_dict["correct"]
                score += 1
            else:
                guess = "Incorrect!"
                correctness_img = util_dict["incorrect"]

            # Display new target emoji
        
            flag = True
            current_emoji = random.choice(emoji_dict.keys())
            i = 100

    elif cv2.waitKey(5) == 27:
        break  
    
    # Add score to the screen
    cv2.putText(frame,"Score: {}".format(score), (450,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),6)
        
    if flag == True:
        cv2.putText(frame,"Detected: {}".format(detected_emotion), (320,300), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),4)
        augment_on_frame(frame, correctness_img, 400, 310, 120, 120)    
    cv2.imshow("Frame", frame)
    
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


-1

### Game - version 2 (Face augmentation)

In [12]:
camera = cv2.VideoCapture(0)
# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
camera.set(cv2.CAP_PROP_FPS, 24)

# Import emoji
emoji_dict = importEmoji()
current_emoji = random.choice(emoji_dict.keys())

# Import utility images
util_dict = importUtil()

# Import haar cascate for face detection
face_cascade = cv2.CascadeClassifier('/usr/local/Cellar/opencv/3.4.0/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')

# Initialize score and counter for displaying result information
i = 0
score = 0

while True:
    i -= 1
    if i <= 0:
        flag = False
    
    
    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    
    # Get grame frame for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces, get coordinate of the left top corner, width and height
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    
#     for detected_face in faces:
#         fx,fy,fw,fh = detected_face
    
    if not isinstance(faces, tuple):
        fx,fy,fw,fh = faces[-1]
 
    # Augment frame with emoji
    emoji_img = emoji_dict[current_emoji]
    augment_on_frame(frame, emoji_img, 10, 10, 150, 150)
    
    if cv2.waitKey(5) == 32:
        # If space bar is pressed, extract imotion
        bytes_string = cv2.imencode('.jpg', frame)[1].tostring()
        result = processRequest(bytes_string)
        
        if result is not None and len(result) > 0:    
            # Detect Emoji
            data8uint = np.fromstring( bytes_string, np.uint8 ) # Convert string to an unsigned int array
            img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
            detected_emotion = getEmotion(result)
                        
            # Check if emoji is matched
            if detected_emotion == current_emoji:
                guess = "Correct!"
                correctness_img = util_dict["correct"]
                score += 1
                previous_emoji = current_emoji
            else:
                guess = "Incorrect!"
                correctness_img = util_dict["incorrect"]
                previous_emoji = None

            # Choose new target emoji
            flag = True
            current_emoji = random.choice(emoji_dict.keys())
            i = 30

    elif cv2.waitKey(5) == 27:
        break  
    
    # Add score to the screen
    cv2.putText(frame,"Score: {}".format(score), (450,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),6)
        
    # Augment the screen 
    if flag == True:
        # Augment previous emoji on the face
        if previous_emoji is not None:
            prev_emoji_img = emoji_dict[previous_emoji]
            augment_on_frame(frame, prev_emoji_img, int(fx-0.2*fw), int(fy-0.2*fh), int(1.4*fw), int(1.4*fh))
        # Add text to the video frame 
        cv2.putText(frame,"Detected: {}".format(detected_emotion), (320,350), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),4)
        augment_on_frame(frame, correctness_img, 400, 360, 120, 120) 

    # Display the final screen
    cv2.imshow("Frame", frame)    
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

SyntaxError: invalid syntax (<ipython-input-12-0a51e5b40848>, line 11)

In [16]:
camera = cv2.VideoCapture(0)
# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
camera.set(cv2.CAP_PROP_FPS, 24)

# Import emoji
emoji_dict = importEmoji()
use_for_test = [5, 2, 7]

em = 0

current_emoji = emoji_dict.keys()[use_for_test[em]]

# Import utility images
util_dict = importUtil()

# Import haar cascate for face detection
face_cascade = cv2.CascadeClassifier('/usr/local/Cellar/opencv/3.4.0/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')

# Initialize score and counter for displaying result information
i = 0
score = 0

while True:
    i -= 1
    if i <= 0:
        flag = False
    
    
    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    
    # Get grame frame for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces, get coordinate of the left top corner, width and height
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    
#     for detected_face in faces:
#         fx,fy,fw,fh = detected_face
    
    if not isinstance(faces, tuple):
        fx,fy,fw,fh = faces[-1]
 
    # Augment frame with emoji
    emoji_img = emoji_dict[current_emoji]
    augment_on_frame(frame, emoji_img, 10, 10, 150, 150)
    
    if cv2.waitKey(5) == 32:
        # If space bar is pressed, extract imotion
        bytes_string = cv2.imencode('.jpg', frame)[1].tostring()
        result = processRequest(bytes_string)
        
        if result is not None and len(result) > 0:    
            # Detect Emoji
            data8uint = np.fromstring( bytes_string, np.uint8 ) # Convert string to an unsigned int array
            img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
            detected_emotion = getEmotion(result)
                        
            # Check if emoji is matched
            if detected_emotion == current_emoji:
                guess = "Correct!"
                correctness_img = util_dict["correct"]
                score += 1
                previous_emoji = current_emoji
            else:
                guess = "Incorrect!"
                correctness_img = util_dict["incorrect"]
                previous_emoji = None

            # Choose new target emoji
            flag = True
            em = (em+1)%3
            current_emoji = emoji_dict.keys()[use_for_test[em]]

            i = 30

    elif cv2.waitKey(5) == 27:
        break  
    
    # Add score to the screen
    cv2.putText(frame,"Score: {}".format(score), (450,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),6)
        
    # Augment the screen 
    if flag == True:
        # Augment previous emoji on the face
        if previous_emoji is not None:
            prev_emoji_img = emoji_dict[previous_emoji]
            augment_on_frame(frame, prev_emoji_img, int(fx-0.2*fw), int(fy-0.2*fh), int(1.4*fw), int(1.4*fh))
        # Add text to the video frame 
        cv2.putText(frame,"Detected: {}".format(detected_emotion), (320,350), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),4)
        augment_on_frame(frame, correctness_img, 400, 360, 120, 120) 

    # Display the final screen
    cv2.imshow("Frame", frame)    
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


-1